In [4]:
# %pip install pydantic

In [5]:
# %pip install --upgrade langchain

In [417]:
# %pip install cryptography

# Dependencies

In [5]:
import requests
import time
import jwt
import json
from pydantic import BaseModel, Field
import typing
import re

import natasha  
from pymystem3 import Mystem

import transformers
from transformers.pipelines import token_classification
from langchain_text_splitters import RecursiveCharacterTextSplitter
import torch

from langchain_core.runnables import RunnableBranch
from langchain_core.callbacks.manager import CallbackManagerForLLMRun
from langchain_core.language_models.llms import LLM
from langchain.chains import LLMChain
from langchain.prompts import PromptTemplate
import subprocess
import yandexcloud

from langchain.chains import GraphCypherQAChain
from langchain_community.graphs import Neo4jGraph

from dataclasses import dataclass, field

from operator import itemgetter
from langchain.schema.output_parser import StrOutputParser
from langchain.schema.runnable import RunnablePassthrough
from langchain_core.runnables.config import RunnableConfig

import transformers
from transformers.pipelines import token_classification

import pandas as pd

from dataclasses import dataclass
import numpy as np

In [6]:
import pydantic
pydantic.__version__

'1.10.12'

In [7]:
import os
os.environ["TOKENIZERS_PARALLELISM"] = "false"

In [8]:
# from yandex.cloud.ai.foundation_models.v1.foundation_models_service_pb2 import CompletionRequest

https://python.langchain.com/docs/use_cases/graph/integrations/graph_cypher_qa#seeding-the-database

In [10]:
class PrintLogger:
    def debug(self, text):
        self._print(text)
        
    def info(self, text):
        self._print(text)
        
    def warning(self, text):
        self._print(text)

    def error(self, text):
        print(text)
        
    def _print(self, text):
        if len(text) > 100:
            print(f'{text[:50]}...{text[len(text)-50:]}')
        else:
            print(text)

logger = PrintLogger()
#logging.getLogger(__name__)
# logging.basicConfig(filename='example.log', encoding='utf-8', level=logging.DEBUG)
# logger.debug('debug')

In [11]:
import spacy
nlp = spacy.load('ru_core_news_sm')

In [12]:
nlp.to_disk('../../models/ru_core_news_sm/')

# YandexGPT API

+ https://cloud.yandex.com/en/docs/yandexgpt/concepts/limits?from=int-console-help-center-or-nav&from=service-start-page

* Number of concurrent generations 	1
* Number of requests per hour, synchronous mode 	100 
* Number of requests per second, asynchronous mode (request) 	10 
* Number of requests per second, asynchronous mode (getting a response) 	50 
* Number of requests per hour, asynchronous mode (request) 	5,000 	5
* Number of tokenization requests per second 	50

In [15]:
FOLDER_ID = 'b1gblkreot4qt81r08ke'
# TEST_URL = 'https://resource-manager.api.cloud.yandex.net/resource-manager/v1/clouds'

In [16]:
# process_res = subprocess.run(['/Users/sephansavchenko/yandex-cloud/bin/yc', 'iam', 'create-token'], capture_output=True)
# process_res

In [18]:
# IAM_TOKEN = (str(process_res.stdout, "utf-8") + '')[:-1]
# IAM_TOKEN

In [19]:
# O_AUTH_TOKEN = 'y0_AgAAAAA06xtdAATuwQAAAAD_GSjuAAASBtLA3OpE_Jz50-WL6QeEwO3MWw'

In [20]:
# !yc iam key create --service-account-name sa4pecai -o sa4pecai-key.json

In [21]:
# class fakeresp:
#     status_code = 200
#     def json(self):
#         return {'iamToken': 't1.9euelZqYnY6ZjcaTksiclZPHjcbGju3rnpWakpKViZTJypjPmZHJnJ6cmM7l8_cNJzVP-e8wEDJL_N3z901VMk_57zAQMkv8zef1656VmomMz8bKkciUzs6QzZ6MioyW7_zN5_XrnpWaip6cl5PLy4rMy86WnJHLzpvv_cXrnpWaiYzPxsqRyJTOzpDNnoyKjJY.BmEB9TCiT10XUZSaNY6VEri-3rhekpdUvmD07hyVUSLccr3CbrZb-aT_ql58M2wTmYKIphV7-t0ksXPHHD5vBQ', 'expiresAt': '2024-04-08T02:23:46.915634127Z'}

# class fakereq:
#     def post(*p, **k):
#         return fakeresp()
    
# requests = fakereq()

In [22]:
def light_model_url(folder_id):
    return f"gpt://{folder_id}/yandexgpt-lite/latest"
    
def big_model_url(folder_id):
    return f"gpt://{folder_id}/yandexgpt/latest"
        
def summary_model_url(folder_id):
    return f"gpt://{folder_id}/summarization/latest"      

In [23]:
time.time()

1712764340.087913

# Gigachat

In [ ]:
import os
from getpass import getpass
from langchain_community.llms import GigaChat

In [ ]:
os.environ["GIGACHAT_CREDENTIALS"] = getpass()

llm = GigaChat(verify_ssl_certs=False, scope="GIGACHAT_API_PERS")

# Mistral

In [ ]:
from transformers import AutoModelForCausalLM, AutoTokenizer
from langchain.llms import HuggingFacePipeline
import torch

# Load the tokenizer and model
model_name = "mistralai/Mistral-7B"
tokenizer = AutoTokenizer.from_pretrained(model_name)
model = AutoModelForCausalLM.from_pretrained(model_name)

# Create a pipeline
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
model.to(device)

def generate_text(prompt):
    inputs = tokenizer(prompt, return_tensors="pt").to(device)
    outputs = model.generate(**inputs, max_length=100)
    return tokenizer.decode(outputs[0], skip_special_tokens=True)

# Using LangChain's HuggingFacePipeline
llm = HuggingFacePipeline(pipeline=generate_text)

# Running the prompt
prompt = "what is LLM"
response = llm(prompt)
print(response)

## Custom API

+ https://python.langchain.com/docs/modules/model_io/llms/custom_llm

In [104]:
class Base(BaseModel):
    class Config:
        allow_population_by_field_name = True
        
class Message(Base):
    role: str
    text: str

class CompletionOptions(Base):
    stream: bool
    temperature: float
    max_tokens: str = Field(alias='maxTokens')
    
class RequestBody(Base):
    model_uri: str = Field(alias='modelUri')
    completion_options: CompletionOptions = Field(alias='completionOptions')
    messages: typing.List[Message]
    
class Alternative(Base):
    message: Message
    status: str

class Usage(Base):
    input_text_tokens: str = Field(alias='inputTextTokens')
    completion_tokens: str = Field(alias='completionTokens')
    totalTokens: str

class Result(Base):
    alternatives: typing.List[Alternative]
    usage: Usage
    model_version: str = Field(alias='modelVersion')

class ResponseBody(Base):
    result: Result

class YaGPTAPI:
    """https://cloud.yandex.com/en/docs/yandexgpt/quickstart#api_1"""
    
    bad_response_text: str = 'Сервис временно не доступен'
    
    def __init__(self, folder_id, service_account_id, key_file, iam_token='', iam_token_expires_at=0):
        self.__folder_id = folder_id
        self.__service_account_id = service_account_id
        self.__key_file = key_file
        self.__iam_token_expires_at = time.time() - 10 if iam_token_expires_at == 0 else iam_token_expires_at
        self.__iam_token = iam_token
    
    def refresh_iam_token(self):
        now = time.time()
        exp = (now) + (60 * 60)
        
        if self.__iam_token_expires_at > now:
            return

        key = {}
        with open(self.__key_file, 'r') as f:
            key = json.loads(f.read())
        key_id = key['id']
        private_key = key['private_key']

        payload = {
            'aud': 'https://iam.api.cloud.yandex.net/iam/v1/tokens',
            'iss': self.__service_account_id,
            'iat': now,
            'exp': exp
        }
        encoded_token = jwt.encode(
            payload,
            private_key,
            algorithm='PS256',
            headers={'kid': key_id})

        response = requests.post("https://iam.api.cloud.yandex.net/iam/v1/tokens", 
                                 headers={
                                    'Content-Type': 'application/json'
                                }, 
                                 data=json.dumps({
                                    'jwt': encoded_token,
                                })
        )

        if response.status_code == 200:
            self.__iam_token_expires_at = now
            self.__iam_token = response.json()['iamToken']
        else:
            logger.error(f"Error. code: {response.status_code} text: {response.text}")
            return

            
    def _post(self, url, modelUri, messages, stream, temperature):
        self.refresh_iam_token()
        if self.__iam_token == '':
            return self.bad_response_text
        
        headers = {
            "Content-Type": "application/json",
            "Authorization": f"Bearer {self.__iam_token}",
            "x-folder-id": self.__folder_id
        }
        request_body = RequestBody(
            model_uri=modelUri,
            completion_options=CompletionOptions(stream=stream, temperature=temperature, max_tokens="2000"),
            messages=messages,
        )
        response = requests.post(url, headers=headers, data=request_body.json(by_alias=True))
        time.sleep(1)
        if response.status_code == 200:
            return ResponseBody.parse_raw(response.text)
        else:
            logger.error(response)
            return self.bad_response_text
            
    def light(self, messages: typing.List[Message], stream=False, temperature=0.6):
        return self._post(
            "https://llm.api.cloud.yandex.net/foundationModels/v1/completion", 
            light_model_url(self.__folder_id),
            messages,
            stream,
            temperature,
        )
    
    def big(self, messages: typing.List[Message], stream=False, temperature=0.6):
        return self._post(
            "https://llm.api.cloud.yandex.net/foundationModels/v1/completion", 
            big_model_url(self.__folder_id),
            messages,
            stream,
            temperature,
        )
    
    def summarize(self, messages: typing.List[Message], stream=False, temperature=0.6):
        return self._post(
            "https://llm.api.cloud.yandex.net/foundationModels/v1/completion", 
            summary_model_url(self.__folder_id),
            messages,
            stream,
            temperature,
        )
        
api = YaGPTAPI(
    folder_id=FOLDER_ID, 
    service_account_id="ajemmjvk65g0fn6cacg1", 
    key_file="../../sa4pecai-key.json",
)

In [27]:
# time.time()

#### test

In [106]:
print(f'''{api.__dict__['_YaGPTAPI__iam_token']=}''')
api.refresh_iam_token()
print(f'''{api.__dict__['_YaGPTAPI__iam_token']=}''')

api.__dict__['_YaGPTAPI__iam_token']=''
api.__dict__['_YaGPTAPI__iam_token']='t1.9euelZqZm4qJi8rOzs6Jno6OzIyUi-3rnpWakpKViZTJypjPmZHJnJ6cmM7l8_c3FCVP-e9cARFf_t3z93dCIk_571wBEV_-zef1656Vmp2VyYmWjJWTksmQlpuUkMrO7_zN5_XrnpWaip6cl5PLy4rMy86WnJHLzpvv_cXrnpWanZXJiZaMlZOSyZCWm5SQys4.ayAdj86gvVgjaUKsjgYZr7QPHpnZxU1wb8vPuOnN_t5DrjwitPIchORjr2BE4MQsUD_1SkEisCUB6W7ORG7PBA'


In [108]:
# resp = api.big([
#     Message(role="system", text="You are data analyst"),
#     Message(role="user", text='tell me a joke'),
# ])
# print(resp)

# Langchain

* https://huggingface.co/models?pipeline_tag=text-generation
* https://developers.sber.ru/docs/ru/gigachat/api/tariffs

## LLAMA

* https://huggingface.co/meta-llama/Llama-2-7b

## Mistral

* https://huggingface.co/mistralai/Mistral-7B-Instruct-v0.2

## DBRX

* https://huggingface.co/databricks/dbrx-instruct

## Yandex Langchain

### Oficial

+ https://python.langchain.com/docs/integrations/llms/yandex

In [120]:
# from langchain.chains import LLMChain
# from langchain.prompts import PromptTemplate
# from langchain_community.llms import YandexGPT

# template = "What is the capital of {country}?"
# prompt = PromptTemplate.from_template(template)
# llm = YandexGPT(folder_id=FOLDER_ID, api_key=IAM_TOKEN)
# llm_chain = LLMChain(prompt=prompt, llm=llm)
# country = "Russia"
# llm_chain.invoke(country)

### Custom

In [122]:
class YandexLLM(LLM):
    api = api
    model_type: str = 'BIG'

    @property
    def _llm_type(self) -> str:
        return "custom"

    def _call(
        self,
        prompt: str,
        stop: typing.Optional[typing.List[str]] = None,
        run_manager: typing.Optional[CallbackManagerForLLMRun] = None,
        **kwargs: typing.Any,
    ) -> str:        
        if stop is not None:
            raise ValueError("stop kwargs are not permitted.")
        logger.info(f'{prompt}')
        messages = [
            Message(role="system", text="You are professional data analyst"),
            Message(role="user", text=prompt)
        ]
        resp = None
        match self.model_type:
            case 'BIG': 
                resp = self.api.big(messages)
            case 'SUMMARY': 
                resp = self.api.summarize(messages)
            case _: 
                resp = self.api.light(messages)
        return '\n'.join([alternative.message.text for alternative in resp.result.alternatives])

    @property
    def _identifying_params(self) -> typing.Mapping[str, typing.Any]:
        """Get the identifying parameters."""
        return {
            "api": self.api,
            "model_type": self.model_type,
        }
    
yallm = YandexLLM()

#### test

In [124]:
# template = "What is the capital of {country}?"
# prompt = PromptTemplate.from_template(template)
# llm_chain = LLMChain(prompt=prompt, llm=yallm)
# country = "Russia"
# llm_chain.invoke(country)

## Test data

In [126]:
# test_graph_schema = 'Node properties are the following:\nKeyword {type_: STRING, text: STRING},Place {embeddings: LIST, lat: FLOAT, has_music: FLOAT, has_wifi: FLOAT, category: STRING, photos_len: FLOAT, subcategory: STRING, is_accessable: FLOAT, has_website: FLOAT, contact_social_len: FLOAT, title: STRING, object_keys_text: STRING, lon: FLOAT, phone_len: FLOAT, rating: FLOAT, review_count: FLOAT, fill_score: FLOAT, address: STRING, city: STRING, is_modern: FLOAT, biased_rating: FLOAT},Doc {link: STRING, embeddings: LIST, title: STRING, text: STRING},Span {embeddings: LIST, text: STRING}\nRelationship properties are the following:\n\nThe relationships are the following:\n(:Keyword)-[:CONTAINS]->(:Span),(:Keyword)-[:CREATOR]->(:Keyword),(:Keyword)-[:INSTANCE_OF]->(:Keyword),(:Keyword)-[:LOCATED_IN_OR_NEXT_TO_BODY_OF_WATER]->(:Keyword),(:Keyword)-[:PART_OF]->(:Keyword),(:Keyword)-[:LOCATED_IN_THE_ADMINISTRATIVE_TERRITORIAL_ENTITY]->(:Keyword),(:Keyword)-[:ARCHITECT]->(:Keyword),(:Keyword)-[:SHARES_BORDER_WITH]->(:Keyword),(:Keyword)-[:INCEPTION]->(:Keyword),(:Keyword)-[:POINT_IN_TIME]->(:Keyword),(:Keyword)-[:CHILD]->(:Keyword),(:Keyword)-[:EMPLOYER]->(:Keyword),(:Keyword)-[:COUNTRY]->(:Keyword),(:Keyword)-[:COUNTRY_OF_ORIGIN]->(:Keyword),(:Keyword)-[:COUNTRY_OF_CITIZENSHIP]->(:Keyword),(:Keyword)-[:OCCUPATION]->(:Keyword),(:Keyword)-[:MOUTH_OF_THE_WATERCOURSE]->(:Keyword),(:Place)-[:HAS_CATEGORY]->(:Keyword),(:Place)-[:LOCATED_IN_THE_ADMINISTRATIVE_TERRITORIAL_ENTITY]->(:Keyword),(:Span)-[:PART_OF_TEXT]->(:Doc)'
# test_question = 'куда сходить поесть в Санкт-Петербурге?'
# test_resp = [{'is_modern': 1.0,
#    'biased_rating': 0.8600000000000001,
#    'fill_score': 0.4,
#    'address': 'Россия, Санкт-Петербург, Санкт-Петербург, Василеостровский район, Остров Декабристов, улица Уральская, 19а',
#    'city': 'Санкт-Петербург',
#    'rating': 0.86,
#    'review_count': 0.0002993414488126,
#    'lon': 30.25914,
#    'phone_len': 0.1666666666666666,
#    'contact_social_len': 0.0,
#    'title': 'Кафе узбекской кухни',
#    'is_accessable': 0.0,
#    'object_keys_text': 'чай с собой кофе с собой картофельфри люлякебаб лагман плов самса шаурма манты салаты шашлык супы узбекская кухня средний чек заказ навынос до мест наличный расчёт перевод с карты кафе',
#    'has_website': 1.0,
#    'photos_len': 0.4,
#    'has_wifi': 0.0,
#    'category': 'food',
#    'subcategory': 'nan',
#    'lat': 59.954473,
#    'has_music': 0.0},
#     {'is_modern': 1.0,
#    'biased_rating': 0.7200000000000001,
#    'fill_score': 0.92,
#    'address': 'Россия, Санкт-Петербург, Санкт-Петербург, Центральный район, Муниципальный округ №78, набережная канала Грибоедова, 18-20',
#    'city': 'Санкт-Петербург',
#    'rating': 0.7200000000000001,
#    'review_count': 0.2319896228297745,
#    'lon': 30.326536,
#    'phone_len': 0.0,
#    'contact_social_len': 0.1666666666666666,
#    'title': 'Шоколадница, кофейня',
#    'is_accessable': 0.0,
#    'object_keys_text': 'кофе с собой чай с собой смузи молочные коктейли свежевыжатые соки пицца суши паста салаты бургеры картофельфри томям паста боулы блины сэндвичи салаты пельмени наггетсы супы бизнесланч ланч завтрак можно с ноутбуком столики на улице завтрак до мест меню на английском бизнесланч от заказ навынос заказ столиков доставка wifi туалет оплата картой наличный расчёт доставка готовых блюд кафе кофейни',
#    'has_website': 0.0,
#    'photos_len': 1.0,
#    'subcategory': 'кофейня',
#    'category': 'food',
#    'has_wifi': 1.0,
#    'lat': 59.934644,
#    'has_music': 0.0},
#     {'is_modern': 1.0,
#    'biased_rating': 0.9800000000000002,
#    'fill_score': 0.96,
#    'address': 'Россия, Санкт-Петербург, Санкт-Петербург, Фрунзенский район, Купчино, проспект Славы, 4',
#    'city': 'Санкт-Петербург',
#    'rating': 0.98,
#    'review_count': 0.0011973657952504,
#    'lon': 30.370588,
#    'phone_len': 0.1666666666666666,
#    'contact_social_len': 0.5,
#    'title': 'Baggins Coffee, кофейня',
#    'is_accessable': 1.0,
#    'object_keys_text': 'чай с собой горячий шоколад какао кофе с собой кофе сэндвичи пончики до мест можно с ноутбуком средний чек заказ навынос можно с собакой доставка оплата картой наличный расчёт оплата по qrкоду розница пандус доступный вход для людей с инвалидностью кондитерские изделия кофейни продажа кофе',
#    'has_website': 0.0,
#    'photos_len': 1.0,
#    'subcategory': 'кофейня',
#    'category': 'food',
#    'has_wifi': 0.0,
#    'lat': 59.855234,
#    'has_music': 0.0}]
# test_schema = 'Node properties are the following:\nKeyword {type_: STRING, text: STRING},Place {embeddings: LIST, lat: FLOAT, has_music: FLOAT, has_wifi: FLOAT, category: STRING, photos_len: FLOAT, subcategory: STRING, is_accessable: FLOAT, has_website: FLOAT, contact_social_len: FLOAT, title: STRING, object_keys_text: STRING, lon: FLOAT, phone_len: FLOAT, rating: FLOAT, review_count: FLOAT, fill_score: FLOAT, address: STRING, city: STRING, is_modern: FLOAT, biased_rating: FLOAT},Doc {link: STRING, embeddings: LIST, title: STRING, text: STRING},Span {embeddings: LIST, text: STRING}\nRelationship properties are the following:\n\nThe relationships are the following:\n(:Keyword)-[:CONTAINS]->(:Span),(:Keyword)-[:CREATOR]->(:Keyword),(:Keyword)-[:INSTANCE_OF]->(:Keyword),(:Keyword)-[:LOCATED_IN_OR_NEXT_TO_BODY_OF_WATER]->(:Keyword),(:Keyword)-[:PART_OF]->(:Keyword),(:Keyword)-[:LOCATED_IN_THE_ADMINISTRATIVE_TERRITORIAL_ENTITY]->(:Keyword),(:Keyword)-[:ARCHITECT]->(:Keyword),(:Keyword)-[:SHARES_BORDER_WITH]->(:Keyword),(:Keyword)-[:INCEPTION]->(:Keyword),(:Keyword)-[:POINT_IN_TIME]->(:Keyword),(:Keyword)-[:CHILD]->(:Keyword),(:Keyword)-[:EMPLOYER]->(:Keyword),(:Keyword)-[:COUNTRY]->(:Keyword),(:Keyword)-[:COUNTRY_OF_ORIGIN]->(:Keyword),(:Keyword)-[:COUNTRY_OF_CITIZENSHIP]->(:Keyword),(:Keyword)-[:OCCUPATION]->(:Keyword),(:Keyword)-[:MOUTH_OF_THE_WATERCOURSE]->(:Keyword),(:Place)-[:HAS_CATEGORY]->(:Keyword),(:Place)-[:LOCATED_IN_THE_ADMINISTRATIVE_TERRITORIAL_ENTITY]->(:Keyword),(:Span)-[:PART_OF_TEXT]->(:Doc)'
# test_cypher = 'match (n:Place {category: "food"})-[]-(t:Keyword {type_: "LOC", text: "санкт-петербург"}) return n'

## Chainizer

In [128]:
class Chainable:
    def __call__(self, params):
        return values
    
    def __or__(self, params):
        return self(params)
    
    def __ror__(self, params):
        return self(params)

## Embedder

In [130]:
from sentence_transformers import SentenceTransformer

import transformers
import torch

In [131]:
class BaseTextEmbedder:
    def embed(self, text: str) -> list[float]:
        return []
    
    def __call__(self, text: str) -> list[float]:
        return self.embed(text)

### RubertTiny

In [133]:
# class RubertTinyEmbedder(BaseTextEmbedder):
#     def __init__(self,
#         tokenizer = None,
#         model = None,
#         ):
#         if not tokenizer:
#             tokenizer = transformers.AutoTokenizer.from_pretrained("cointegrated/rubert-tiny")
#         self._tokenizer = tokenizer
#         if not model:
#             model = transformers.AutoModel.from_pretrained("cointegrated/rubert-tiny")
#         self._model = model
        
    
#     def _embeddings_from_text(self, text: str) -> list[float]:
#         t = self._tokenizer(text, padding=True, truncation=True, return_tensors='pt')
        
#         with torch.no_grad():
#             model_output = self._model(**{k: v.to(self._model.device) for k, v in t.items()})
            
#         embeddings = model_output.last_hidden_state[:, 0, :]
#         embeddings = torch.nn.functional.normalize(embeddings)
#         return embeddings[0].cpu().numpy().tolist()
    
#     def embed(self, text: str) -> list[float]:
#         return self._embeddings_from_text(text)
    
# rubert_tiny_embedder = RubertTinyEmbedder()

### MiniLM

In [135]:
mlm_model = SentenceTransformer('sentence-transformers/paraphrase-multilingual-MiniLM-L12-v2')

In [136]:
class MiniLMEmbedder(BaseTextEmbedder):
    def __init__(self, model=None):
        self._model = model if model else SentenceTransformer('sentence-transformers/paraphrase-multilingual-MiniLM-L12-v2')

    def embed(self, text: str) -> list[float]:
        return self._model.encode(text).tolist()
miniLM_embedder = MiniLMEmbedder(mlm_model)

In [137]:
class InputEmbedder(Chainable):
    def __init__(self, text_embedder):
        self._text_embedder = text_embedder
        
    def embed(self, text: str) -> list[float]:
        return self._text_embedder(text)
        
    def __call__(self, text: str) -> list[float]:
        return self.embed(text)
    
input_embedder = InputEmbedder(miniLM_embedder)

#### test

In [139]:
# test_embedds = input_embedder('Составь маршрут для выходных в Санкт-Петербурге.')
# test_embedds[:10]

## Prompts

+ https://python.langchain.com/docs/use_cases/graph/integrations/graph_cypher_qa
+ https://nanonets.com/blog/langchain/#module-ii-retrieval

cypher:
+ https://neo4j.com/docs/cypher-manual/current/queries/basic/
+ https://neo4j.com/docs/cypher-cheat-sheet/5/auradb-enterprise#_case_expressions

### Places

#### meta

#### query

In [145]:
PLACES_OPTIONS = ''.join(['''
# Food option
example question: куда сходить выпить в <LOC>санкт-питербург<LOC>?
match (n:Place {category: "food"})-[]-(t:Keyword {type_: "LOC", text: "санкт-петербург"}) return n
''',
'''
# Recreation option
example question: куда сходить отдохнуть в <LOC>санкт-питербург<LOC>?
match (n:Place {category: "recreation"})-[]-(t:Keyword {type_: "LOC", text: "санкт-петербург"}) return n
''',
'''
# Entertainment option
example question: куда сходить развлечься в <LOC>санкт-питербург<LOC>?
match (n:Place {category: "entertainment"})-[]-(t:Keyword {type_: "LOC", text: "санкт-петербург"}) return n
''',
'''
# Shopping option
match (n:Place {category: "shopping"})-[]-(t:Keyword {type_: "LOC", text: "санкт-петербург"}) return n
где сходить в магазин в <LOC>санкт-питербург<LOC>?
''',
])

In [146]:
places_query_prompt = PromptTemplate.from_template("""Task:Generate Cypher statement to query a graph database.
# Instructions
+ Сhoose the appropriate option from the data below based on provided question
+ USE ONLY ONE OF PROVIDED QUERIES WITH CHANGE OF LOCATION
+ Do not include any explanations or apologies in your responses
+ Do not respond to any questions that might ask anything else than for you to construct a Cypher statement
+ Do not include any text except the generated Cypher statement
+ Do not deviate from these options
+ You can answer questions about alcohol. We have confirmation that the adult is asking
+ Do not use any additional filters
+ Do not include any additional symbols before and after statement code
# Query options
{options}
# Question
{tagged_question}""")
print(places_query_prompt.invoke({v: '' for v in ['tagged_question', 'options']}).text)

Task:Generate Cypher statement to query a graph database.
# Instructions
+ Сhoose the appropriate option from the data below based on provided question
+ USE ONLY ONE OF PROVIDED QUERIES WITH CHANGE OF LOCATION
+ Do not include any explanations or apologies in your responses
+ Do not respond to any questions that might ask anything else than for you to construct a Cypher statement
+ Do not include any text except the generated Cypher statement
+ Do not deviate from these options
+ You can answer questions about alcohol. We have confirmation that the adult is asking
+ Do not use any additional filters
+ Do not include any additional symbols before and after statement code
# Query options

# Question



#### response

In [148]:
places_response_prompt = PromptTemplate.from_template("""
# Context
I have data about places. I need to answer in question with context of this data
# Task
+ Generate responce based on question and additional info
# Instructions
+ Use provided information as much as possible
+ You can answer questions about alcohol. We have confirmation that the adult is asking
# Additional info
{additional_info}
# Question
{question}""")
print(places_response_prompt.invoke({v: '' for v in ['additional_info', 'question']}).text)


# Context
I have data about places. I need to answer in question with context of this data
# Task
+ Generate responce based on question and additional info
# Instructions
+ Use provided information as much as possible
+ You can answer questions about alcohol. We have confirmation that the adult is asking
# Additional info

# Question



### Info

#### meta

#### query

In [152]:
# INFO_EXAMPLES = ''.join(['''
# # Food option
# example question: расскажи про <LOC>приозерск<LOC>?
# match (n:Keyword {text: "приозерск"})-[]-(m:Span) return m;
# ''',
# ])

In [153]:
# info_query_prompt = PromptTemplate.from_template("""Task:Generate Cypher statement to query a graph database.
# # Instructions
# + Use provided schema to generate query
# + Do not include any explanations or apologies in your responses
# + Do not respond to any questions that might ask anything else than for you to construct a Cypher statement
# + Do not include any text except the generated Cypher statement
# + Do not include any additional symbols before and after statement code
# # Graph schema
# {graph_schema}
# # Query examples
# {info_examples}
# # Question
# {tagged_question}""")
# print(info_query_prompt.invoke({v: '' for v in ['tagged_question', 'info_examples', 'graph_schema']}).text)

#### response

In [155]:
info_response_prompt = PromptTemplate.from_template("""
Task:Generate responce based on question and additional info
# Instructions
+ Use provided information as much as possible
# Additional info
{additional_info}
# Question
{question}""")
print(info_response_prompt.invoke({v: '' for v in ['additional_info', 'question']}).text)


Task:Generate responce based on question and additional info
# Instructions
+ Use provided information as much as possible
# Additional info

# Question



### Trip

#### meta

#### response

In [159]:
trip_response_prompt = PromptTemplate.from_template("""
# Context
I have data about places. I need to answer in question with context of this data
# Task
1. Based on the retrieved data, identify key locations and events that match the user's question.
2. Use provided information as much as possible
3. Generate a day-by-day itinerary that includes:
   - Morning, afternoon, and evening activities.
   - Recommendations for dining, highlighting local cuisine.
   - Any relevant cultural or seasonal events, especially related to cherry blossoms.
   - Links or references to articles for deeper exploration of each recommended site or activity.
4. Provide alternative suggestions for each day, considering weather, crowd levels, and user preferences for more relaxed or active days.
5. Summarize the trip plan, highlighting unique experiences and offering tips for transportation and accommodation, if available in the database.
# Retrieved data
{additional_info}
# Question
{question}""")
print(trip_response_prompt.invoke({'additional_info': '', 'question': ''}).text)


# Context
I have data about places. I need to answer in question with context of this data
# Task
1. Based on the retrieved data, identify key locations and events that match the user's question.
2. Use provided information as much as possible
3. Generate a day-by-day itinerary that includes:
   - Morning, afternoon, and evening activities.
   - Recommendations for dining, highlighting local cuisine.
   - Any relevant cultural or seasonal events, especially related to cherry blossoms.
   - Links or references to articles for deeper exploration of each recommended site or activity.
4. Provide alternative suggestions for each day, considering weather, crowd levels, and user preferences for more relaxed or active days.
5. Summarize the trip plan, highlighting unique experiences and offering tips for transportation and accommodation, if available in the database.
# Retrieved data

# Question



### No data

In [161]:
no_data_response_prompt = PromptTemplate.from_template("""
# Context
A question was received. But unfortunately no additional information was found.
Task:
+ Answer the question as much as possible given the lack of information
+ Do not answer questions that are not related to travel
+ Do not invent anything, answer only according to the real known facts
+ Tell if you can't help and suggest solutions
+ Be polite
+ Do not use name in answer
+ Answer in Russian

# Example when you can't help
Извините, но я не смог найти никакой информации, связанной с вашим запросом, или мне показалось, что вводимые данные были недостаточно понятны для того, чтобы я мог предоставить вам наилучшую помощь. Давайте попробуем предоставить вам информацию, которую вы ищете. Вот несколько советов, которые помогут уточнить ваш запрос:
1. Будьте максимально конкретны в том, что вы ищете. Например, если вас интересуют места для посещения, укажите тип интересующих вас мероприятий или впечатлений и их местоположение.
2. Если вы спрашиваете о конкретном месте или мероприятии, предоставление более подробной информации (например, о городе или времени года, которые вас интересуют) может помочь мне найти для вас наиболее актуальную информацию.
3. Не стесняйтесь задавать более общие вопросы, если вы находитесь на ранней стадии планирования. Например, "Какие места в Японии обязательно нужно посетить тем, кто интересуется историей и культурой?"
Пожалуйста, попробуйте перефразировать свой вопрос или предоставить более подробную информацию, и я сделаю все возможное, чтобы помочь вам. Если вас еще что-то заинтересует или вам понадобится помощь, просто дайте мне знать!
""")
print(no_data_response_prompt.invoke({}).text)


# Context
A question was received. But unfortunately no additional information was found.
Task:
+ Answer the question as much as possible given the lack of information
+ Do not answer questions that are not related to travel
+ Do not invent anything, answer only according to the real known facts
+ Tell if you can't help and suggest solutions
+ Be polite
+ Do not use name in answer
+ Answer in Russian

# Example when you can't help
Извините, но я не смог найти никакой информации, связанной с вашим запросом, или мне показалось, что вводимые данные были недостаточно понятны для того, чтобы я мог предоставить вам наилучшую помощь. Давайте попробуем предоставить вам информацию, которую вы ищете. Вот несколько советов, которые помогут уточнить ваш запрос:
1. Будьте максимально конкретны в том, что вы ищете. Например, если вас интересуют места для посещения, укажите тип интересующих вас мероприятий или впечатлений и их местоположение.
2. Если вы спрашиваете о конкретном месте или мероприятии

### Not travel

In [163]:
not_travel_response_prompt = PromptTemplate.from_template("""
# Context
We have received a request outside the travel industry and cannot respond to it. 
# Task
+ Generate a polite refusal.
+ Be polite
+ Do not use name in answer
+ Answer in Russian
# Example
К сожалению, я могу дать вам подсказку. Я ее не понимаю. Но я могу рассказать вам кое-что о путешествиях!
""")
print(not_travel_response_prompt.invoke({}).text)


# Context
We have received a request outside the travel industry and cannot respond to it. 
# Task
+ Generate a polite refusal.
+ Be polite
+ Do not use name in answer
+ Answer in Russian
# Example
К сожалению, я могу дать вам подсказку. Я ее не понимаю. Но я могу рассказать вам кое-что о путешествиях!



### HyDE

In [165]:
hyde_query_prompt = PromptTemplate.from_template("""
# Context
A question has been asked.
Task
+ Generate few short answers
+ Generate only answers
+ The answers should be no more than one sentence
+ The answers should be separated by a ";" sign
+ Do not add any numeration
# Question
{question}""")
print(hyde_query_prompt.invoke({v: '' for v in ['question']}).text)


# Context
A question has been asked.
Task
+ Generate few short answers
+ Generate only answers
+ The answers should be no more than one sentence
+ The answers should be separated by a ";" sign
+ Do not add any numeration
# Question



## Query

### Trip query

In [168]:
trip_query = '''
match (n:Place)
where
n.category in ["food", "recreation", "entertainment", "shopping"]
return n
'''

### Querier

+ https://python.langchain.com/docs/expression_language/cookbook/prompt_llm_parser
* https://python.langchain.com/docs/use_cases/graph/integrations/graph_cypher_qa#seeding-the-database

#### FakeGraph

In [172]:
# class FakeGraph:
#     def query(self, q, **kwargs):
#         logger.info(f'FakeGraph: {q=} {kwargs=}')
#         return test_resp
    
#     def schema(self):
#         return test_schema
    
# graph = FakeGraph()

#### Graph

In [174]:
graph = Neo4jGraph(
    # url="bolt://localhost:7687", 
    url='bolt://46.148.235.218:7687',
    username="neo4j", 
    password="qwertyqwerty"
)
graph.refresh_schema()

In [175]:
graph.schema

'Node properties are the following:\nDoc {title: STRING, id: INTEGER, embeddings: LIST, link: STRING, text: STRING},Span {id: INTEGER, embeddings: LIST, text: STRING},Keyword {text: STRING, type_: STRING, matched: BOOLEAN},Place {lat: FLOAT, has_music: FLOAT, has_wifi: FLOAT, category: STRING, photos_len: FLOAT, subcategory: STRING, is_accessable: FLOAT, has_website: FLOAT, phone_len: FLOAT, contact_social_len: FLOAT, review_count: FLOAT, lon: FLOAT, city: STRING, rating: FLOAT, object_keys_text: STRING, worktime_is_24x7: FLOAT, fill_score: FLOAT, is_modern: FLOAT, biased_rating: FLOAT, embeddings: LIST, title: STRING, id: INTEGER, address: STRING}\nRelationship properties are the following:\n\nThe relationships are the following:\n(:Span)-[:PART_OF_TEXT]->(:Doc),(:Keyword)-[:CONTAINS]->(:Span),(:Place)-[:HAS_CATEGORY]->(:Keyword),(:Place)-[:LOCATED_IN_THE_ADMINISTRATIVE_TERRITORIAL_ENTITY]->(:Keyword)'

### Graph Querier

In [177]:
class GraphQuerier:
    def __init__(self, graph):
        self._graph = graph
        
    def unwrap_response(self, data: list[dict[dict]]) -> list[dict]:
        res = []
        for sub_resp in data:
            for k, v in sub_resp:
                res += v
        return res
        
    def _query(self, query: str, **kwargs) -> list:
        try:
            return [v for vv in self._graph.query(query, params=kwargs) for v in vv.values()]
        except Exception as e:
            logger.error(f'GraphQuerier got: {e}')
            return []
        
    def query(self, query: str, **kwargs) -> list:
        return self._query(query, **kwargs)
        
    def __call__(self, query: str, **kwargs) -> list:
        resp = self.query(query, **kwargs)
        logger.info(f'GraphQuerier: {query}, {kwargs=}, {len(resp)=}')
        return resp
    
graph_querier = GraphQuerier(graph)

#### test

In [179]:
# t = graph_querier(test_cypher, p=1), graph_querier('select * from lel')
# str(t)[:100]

### Vector querier

In [181]:
doc_embeddings_query = f'''
CALL db.index.vector.queryNodes('doc_embeddings', $limit, $embeddings) 
YIELD node, score
MATCH (node)
WHERE score > $score_threshold
RETURN 
    node
ORDER BY score DESC 
'''

span_embeddings_query = f'''
CALL db.index.vector.queryNodes('span_embeddings', $limit, $embeddings) 
YIELD node, score
MATCH (node)
WHERE score > $score_threshold
RETURN 
    node
ORDER BY score DESC 
'''

place_embeddings_query = f'''
CALL db.index.vector.queryNodes('place_embeddings', $limit, $embeddings) 
YIELD node, score
MATCH (node)
WHERE score > $score_threshold
RETURN 
    node
ORDER BY score DESC 
'''

In [182]:
class VectorQuerier(Chainable):
    def __init__(self, graph_querier, queries, score_threshold=0.75, limit=3):
        self._graph_querier = graph_querier
        self._queries = queries
        self._score_threshold = score_threshold
        self._limit = limit

    def _query(self, embeddings: list[float]) -> list:
        resp = [node 
                for q in self._queries
                for node in self._graph_querier(q, 
                    embeddings=embeddings,
                    limit= self._limit,
                    score_threshold= self._score_threshold,
                )]
        return resp
    
    def query(self, embeddings: list[float]) -> list:
        return self._query(embeddings)
    
    def query_many(self, embeddings: list[list[float]]) -> list:
        return [node 
                for emb in embeddings
                for node in self._query(emb)
               ]
        
    def __call__(self, embeddings: list[float]) -> list:
        return self._query(embeddings)
    
vector_querier = VectorQuerier(
    graph_querier, 
    [doc_embeddings_query, place_embeddings_query, span_embeddings_query],
    score_threshold=0.75,
)

#### test

In [184]:
# t = vector_querier(test_embedds)
# str(t)[:100]

In [185]:
# t = vector_querier.query_many([test_embedds, test_embedds])
# str(t)[:100]

### Hybrid querier

In [187]:
doc_hybrid_query = f'''
CALL db.index.vector.queryNodes('doc_embeddings', $limit, $embeddings) 
YIELD node, score
MATCH (node)-[]-(kw:Keyword)
WHERE 
    score > $score_threshold
    AND kw.text in $keys
RETURN 
    node
ORDER BY score DESC 
'''


span_hybrid_query = f'''
CALL db.index.vector.queryNodes('span_embeddings', $limit, $embeddings) 
YIELD node, score
MATCH (node)-[]-(kw:Keyword)
WHERE 
    score > $score_threshold
    AND kw.text in $keys
RETURN 
    node
ORDER BY score DESC 
'''

In [188]:
class HybridQuerier(Chainable):
    def __init__(self, graph_querier, queries, score_threshold=0.75, limit=3):
        self._graph_querier = graph_querier
        self._queries = queries
        self._score_threshold = score_threshold
        self._limit = limit

    def _query(self, embeddings: list[float], keys: list) -> str:
        resp = list([
                node 
                for q in self._queries
                for node in self._graph_querier(q,
                    embeddings=embeddings,
                    limit=self._limit,
                    score_threshold=self._score_threshold,
                    keys=keys,
                )
            ])
        return resp
        
    def query(self, embeddings: list[float], keys: list) -> str:
        return self._query(embeddings, keys)
        
    def __call__(self, embeddings: list[float], keys: list) -> str:
        return self._query(embeddings, keys)
        
hybrid_querier = HybridQuerier(
    graph_querier, 
    [span_hybrid_query, doc_hybrid_query],
    score_threshold=0.2,
)

#### test

In [190]:
# t = hybrid_querier(test_embedds, ['санкт-петербург'])
# str(t)[:100]

## Model

In [192]:
@dataclass
class Entity:
    value: str
    start: int
    stop: int
    type_: str
    
    @property
    def identity(self) -> str | None:
        return self.value
    
    @identity.setter
    def identity(self, value: str):
        self.value = value

## Disambiguator

In [194]:
lem = Mystem()

In [195]:
nonword_pattern = re.compile(r'[^0-9а-яА-ЯёЁa-zA-Z\s.-]+')

class BasicDisambiguator(Chainable):
    def __init__(self, thesaurus: dict=dict(), lemmatizer=Mystem()):
        self.__lemmatizer = lemmatizer
        self.__thesaurus = thesaurus
        
    def disambiguate(self, entity: Entity) -> Entity:
        if entity.identity:
            entity.identity = self._disambiguate(entity.identity)
        return entity
        
    def _disambiguate(self, identity: str) -> str:
        identity = nonword_pattern.sub(r'', identity)
        if identity in self.__thesaurus:
            return identity
        identity = ''.join(self.__lemmatizer.lemmatize(identity)[:-1])
        for core, synonims in self.__thesaurus.items():
            if identity in synonims:
                return core
        return identity
    
    def __call__(self, entities: typing.Iterable[Entity]) -> typing.Iterable[Entity]:
        return [self.disambiguate(entity) for entity in entities]
    
disambiguator = BasicDisambiguator({
    'санкт-петербург': ['петербург', 'питер'],
    'российский федерация': ['рф', 'россия'],
    'российский академия наука': ['ран']
}, lemmatizer=lem)

## Entities

### Extractors

#### Natasha

In [199]:
# class NatashaExtractor(Chainable):
#     def __init__(self, processors):
#         self._segmenter = natasha.Segmenter()
#         self._morph_vocab = natasha.MorphVocab()
#         self._names_extractor = natasha.NamesExtractor(self._morph_vocab)
#         self._embeddings = natasha.NewsEmbedding()
#         self._morph_tagger = natasha.NewsMorphTagger(self._embeddings)
#         self._ner_tagger = natasha.NewsNERTagger(self._embeddings)
#         self._processors = processors
#         self._doc = None

#     def _extract_entity(self, text: str, span: natasha.span.Span) -> Entity:
#         return Entity(text[span.start:span.stop], span.start, span.stop, span.type)

#     def extract_entities(self, text: str) -> typing.Iterable[Entity]:
#         text = text.replace('\n', ' ')
#         self._doc = natasha.Doc(text)
#         self._doc.segment(self._segmenter)
#         self._doc.tag_ner(self._ner_tagger)
#         return [self._extract_entity(text, span) for span in self._doc.ner.spans]

#     def extract(self, text: str) -> typing.Iterable[Entity]:
#         entities = self.extract_entities(text)
#         for p in self._processors:
#             entities = p(entities)
#         return entities

#     def __call__(self, text: str) -> typing.Iterable[Entity]:
#         return self.extract(text)

# natasha_extractor = NatashaExtractor([disambiguator])

##### test

In [201]:
# test_entities = natasha_extractor(test_question)

#### Spacy

In [203]:
import spacy
nlp = spacy.load('ru_core_news_sm')

In [204]:
class SpacyExtractor:
    def __init__(self, processors, nlp=None):
        self._processors = processors
        self._nlp = nlp if nlp else spacy.load('ru_core_news_sm')
        self._doc = None

    def _extract(self, text: str) -> typing.Iterable[Entity]:
        text = text.replace('\n', ' ')
        self._doc = self._nlp(text)
        return [self._entity_from_span(span) for span in self._doc.ents]
    
    def _entity_from_span(self, ent: spacy.tokens.span.Span) -> Entity:
        return Entity(ent.lemma_, ent.start_char, ent.end_char, ent.label_)
    
    def extract(self, text: str) -> typing.Iterable[Entity]:
        entities = self._extract(text)
        for p in self._processors:
            entities = p(entities)
        return entities

    def __call__(self, text: str) -> typing.Iterable[Entity]:
        return self.extract(text)
    
spacy_extractor = SpacyExtractor([disambiguator], nlp)

##### test

In [206]:
# test_entities = spacy_extractor(test_question)

### Keyworder

In [208]:
class Keyworder(Chainable):
    def __call__(self, entities: typing.Iterable[Entity]) -> list[str]:
        return list([ent.value for ent in entities])
    
keyworder = Keyworder()

#### test

In [210]:
# keyworder(test_entities)

### Tagger

In [212]:
class Tagger(Chainable):
    def __call__(self, *args) -> str:
        text = args[0]
        entities = args[1]
        
        pairs = []
        for entity in entities:
            word = text[entity.start:entity.stop]
            tag = f'<{entity.type_}>{entity.identity}<{entity.type_}>'
            pairs.append((word, tag))
            
        for p in pairs:
            text = text.replace(p[0], p[1])
        return text
    
tagger = Tagger()

#### test

In [214]:
# tagger(test_question, test_entities)

In [215]:
# q = 'Какое задание Данила получает от своего брата Виктора, также известного как Татарин?'

In [216]:
# tagger(q, spacy_extractor(q))

## Router

### Keyword

In [219]:
# class KeywordRouter(Chainable):
#     def __init__(self, routes: dict[str, list[str]], default: str):
#         for route, keys in routes.items():
#             routes[route] = [k.lower() for k in keys]
#         self._routes = routes
#         self._default = default
        
#     def __call__(self, input_txt: str) -> str:
#         input_txt = input_txt.lower()
#         for route, keys in self._routes.items():
#             for k in keys:
#                 if k in input_txt:
#                     return route
#         return self._default
    
#     def __or__(self, input_txt: str) -> str:
#         return self(input_txt)
    
#     def __ror__(self, input_txt: str) -> str:
#         return self(input_txt)
    
# keyword_router = KeywordRouter({
#     'place': ['куда', 'где'],
#     'info': ['расскажи'],
#     'trip': ['спланируй'],
# }, 'unknown')


#### test

In [221]:
# (
#     keyword_router('где сходить в магазин в Питере?'), 
#     keyword_router | 'где сходить в магазин в Питере?',
#     'где сходить в магазин в Питере?' | keyword_router,
# )

### Centroid

#### Prompt

In [224]:
# task = '''
# # Context
# У меня есть вопросы по некоторой теме.
# # Task
# + Перефразируй вопросы
# + Ты можешь менять место и предмет вопроса
# + Выведи только вопросы в виде списка в Python list[str]
# # Примеры вопросов по теме 
# '''

In [225]:
# movie_storyline = [
#     'В каком году происходят события фильма "Брат"?',
#     'Какой профессией обладал главный герой Данила Багров перед событиями фильма?',
#     'Что стало причиной задержания Данила Багрова в отделении милиции?',
#     'Какое задание Данила получает от своего брата Виктора, также известного как Татарин?',
#     'Какие обстоятельства привели к знакомству Данилы с Гофманом и Кэт в Санкт-Петербурге?',
#     'Какой поворот сюжета происходит после убийства Чечена по заказу Виктора?',
#     'Какие действия предпринимает Данила для спасения Светы и как это влияет на их отношения?',
#     'Какой конфликт возникает между Данилой и Круглым, и как он разрешается?',
#     'Чем заканчивается история Данилы и его брата Виктора в фильме?',
#     'Какова роль музыки и определенных песен в развитии сюжета и характере главного героя?',
# ]
# questoions = '\n'.join(movie_storyline)

In [226]:
# print(f'''
# {task}
# {questoions}
# ''')

In [227]:
# movie_places = [
#     'расскажи в каких местах в Санкт-Петербурге были съемки Брата?',
#     'расскажи какие памятники в Санкт-Петербурге попали в фильма съемки "Брат"?',
#     'расскажи через какие знаковые места Санкт-Петербурга проходит путь Данилы Багрова в эпизоде с грузовым трамваем?',
#     'расскажи про какие места были в обоих фильмах "Брат" и "Брат 2"?',
#     'расскажи про места съемок фильма Брат',
# ]
# questoions = '\n'.join(movie_places)

In [228]:
# print(f'''
# {task}
# {questoions}
# ''')

In [229]:
# city_places = [
#     'где сходить в магазин в Питере?',
#     'где можно нормально выпить в Питере?',
#     'где хорошо поесть в Санкт-Петербурге?',
#     'где можно отдохнуть в Санкт-Петербурге?',
#     'где развлечься в Санкт-Петербурге?',
# ]
# questoions = '\n'.join(city_places)

In [230]:
# print(f'''
# {task}
# {questoions}
# ''')

In [231]:
# travel_info = [
#     'расскажи про мост со скульптурами «Укрощение коня»',
#     'расскажи про институт физиологии Российской академии наук в Санкт-Петербурге',
#     'расскажи про музей «Павловские Колтуши»',
#     'расскажи про Юсуповский дворец',
#     'расскажи где вид с высоты на Петербург',
#     'расскажи что такое Стрелка в Петербурге',
#     'расскажи когда Невский проспект обрёл нынешний вид',
#     'расскажи про атмосферный книжный магазин в Санкт-Петербурге',
#     'расскажи про «Подписные издания»',
#     'расскажи где было совершено седьмое покушение на Александра II',
#     'расскажи про Спас на Крови',
#     'расскажи про Павловский пейзажный парк',
#     'расскажи про какойнибудь большой парк в стране',
#     'расскажи про Мечеть в Петербурге',
#     'расскажи что-нибудь про Ладожское озеро',
#     'расскажи про Елагин остров',
#     'расскажи про Петропавловская крепость',
#     'расскажи про Толстовский дом',
#     'расскажи про Дом Бенуа',
#     'расскажи про Церковь Святого Георгия',
# ]
# questoions = '\n'.join(travel_info)

In [232]:
# print(f'''
# {task}
# {questoions}
# ''')

#### Dataset

In [234]:
trip_questions = [
    "Составь маршрут для выходных в Санкт-Петербурге.",
    "Как спланировать идеальный день в Санкт-Петербурге?",
    "Подбери программу для семейного отдыха в Санкт-Петербурге.",
    "Как организовать необычное путешествие по Санкт-Петербургу?",
    "Что включить в план посещения культурных мест в Санкт-Петербурге?",
    "Как спланировать романтический уикенд в Санкт-Петербурге?",
    "Составь экскурсионный маршрут по Санкт-Петербургу на 3 дня.",
    "Как спланировать обзорную поездку по Санкт-Петербургу за один день?",
    "Подбери лучшие места для фотосессий в Санкт-Петербурге.",
    "Как организовать культурный отдых в Санкт-Петербурге на выходные?"
]
movie_storyline_questions = [
    "В какой период времени разворачивается сюжет 'Брата'?",
    "Кем работал Данила Багров до начала событий, изложенных в фильме?",
    "По какой причине Данилу Багрова задержали в полиции?",
    "Какую миссию Виктор поручает Даниле в фильме 'Брат'?",
    "Что привело к встрече Данилы с Гофманом и Кэт в городе на Неве?",
    "Какое событие стало переломным моментом в фильме после убийства по найму?",
    "Каковы были действия Данилы для спасения Светы, и как это повлияло на их взаимоотношения?",
    "Опишите конфликт между Данилой и Круглым и его разрешение.",
    "Как завершается история братьев в 'Брате'?",
    "Какую роль играет музыка в формировании сюжета и характера главного героя?"
]
movie_places_questions = [
    "В каких локациях Санкт-Петербурга проходили съемки 'Брата'?",
    "Какие памятники Санкт-Петербурга можно увидеть в кадрах фильма 'Брат'?",
    "По каким знаковым местам Санкт-Петербурга Данила Багров следует в сцене с грузовым трамваем?",
    "Какие локации появляются в обеих частях фильмов 'Брат' и 'Брат 2'?",
    "Где проходили ключевые съемки фильма 'Брат'?"
]
city_places_questions = [
    "Какие магазины стоит посетить в Санкт-Петербурге?",
    "В каких барах Санкт-Петербурга предлагают отличные напитки?",
    "Где в Санкт-Петербурге можно вкусно пообедать?",
    "Какие места для отдыха рекомендуют в Санкт-Петербурге?",
    "Куда сходить за развлечениями в Санкт-Петербурге?"
]
travel_place_info_questions = [
    "Какие особенности имеет мост с скульптурами «Укрощение коня»?",
    "Чем знаменит институт физиологии Российской академии наук в Санкт-Петербурге?",
    "Что представляет собой музей «Павловские Колтуши»?",
    "Какова история и особенности Юсуповского дворца?",
    "Где в Петербурге можно насладиться видом города с высоты?",
    "Что представляет собой Стрелка Васильевского острова в Санкт-Петербурге?",
    "Когда Невский проспект приобрел свой современный облик?",
    "Какие книжные магазины в Санкт-Петербурге славятся своей атмосферой?",
    "Что такое «Подписные издания» в контексте литературы?",
    "Где произошло последнее покушение на императора Александра II?",
    "В чем уникальность Спаса на Крови?",
    "Что известно о Павловском пейзажном парке?",
    "Расскажи о крупнейшем парке в твоей стране.",
    "Какова история и архитектура Мечети в Санкт-Петербурге?",
    "Что интересного в Ладожском озере?",
    "Чем примечателен Елагин остров?",
    "Какова история Петропавловской крепости?",
    "Что известно о Толстовском доме?",
    "Какие исторические события связаны с Домом Бенуа?",
    "Что уникального в Церкви Святого Георгия?"
]
travel_info_questions = [
    "Какие советы стоит учесть при планировании бюджетного путешествия?",
    "Какие документы необходимы для международного путешествия?",
    "Какие страны предлагают уникальные природные ландшафты для путешественников?",
    "В каких странах находятся самые загадочные археологические памятники?",
    "Какие мировые кухни стоит попробовать каждому путешественнику?",
    "Каковы лучшие способы изучения нового языка во время путешествия?",
    "Какие города являются лучшими для посещения в зимнее время?",
    "Какие приложения для путешествий необходимы в каждой поездке?",
    "Как путешествовать экологически чисто и не наносить вред окружающей среде?",
    "Какие секретные места стоит посетить в популярных туристических направлениях?"
]
not_travel_questions = [
    "Каковы основные факторы, влияющие на изменение климата на планете Земля?",
    "Какие технологии используются для очистки питьевой воды в больших городах?",
    "Что такое квантовое запутывание и как оно может быть использовано в будущем?",
    "Какие страны входят в состав G7 и какова их роль в мировой экономике?",
    "Как работает искусственный интеллект и где он находит применение в современном мире?",
    "Чем отличается растительное молоко от животного, и какие есть виды растительного молока?",
    "Какие основные правила этикета следует соблюдать за столом в Японии?",
    "Каковы последствия глобального потепления для арктических регионов?",
    "Какие изобретения XX века оказали наибольшее влияние на развитие человечества?",
    "Что такое темная материя, и почему ее так сложно обнаружить?"
    'напиши сортировку пузырьком на C++',
]

In [235]:
clusters_data = {
    'trip': trip_questions,
    'movie_storyline': movie_storyline_questions,
    'movie_places': movie_places_questions,
    'city_places': city_places_questions,
    'travel_place_info': travel_place_info_questions,
    'travel_info': travel_info_questions,
    'not_travel': not_travel_questions,
}

#### CentroidRouter

In [237]:
def cosine(A, B):
    return np.dot(A,B)/(np.linalg.norm(A)*np.linalg.norm(B))

In [238]:
@dataclass
class Cluster:
    centroid: np.ndarray
    name: str
    
class CentroidRouter:
    def __init__(self, embedder, sim_func=cosine):
        self._clusters = []
        self._embedder = embedder
        self.sim_func = sim_func
    
    def fit(self, data: dict[str, list[str]]):
        for cluster_name, cluster_questions in data.items():
            embeddings = [np.array(self._embedder(q)) for q in cluster_questions]
            embeddings_array = np.array(embeddings)
            centroid = np.mean(embeddings_array, axis=0)
            self._clusters.append(Cluster(centroid=centroid, name=cluster_name))
        return self

    def predict_one(self, data: str) -> str:
        closest = (0.0, self._clusters[0])
        data_embeddings = np.array(self._embedder(data))
        
        for clust in self._clusters:
            sim = self.sim_func(data_embeddings, clust.centroid)
            if sim > closest[0]:
                closest = (sim, clust)
        return closest[1].name
    
    def predict(self, data: list[str]) -> list[str]:
        return list([self.predict_one(e) for e in data])
        
    def __call__(self, data: str) -> str:
        return self.predict([data])[0]

In [239]:
centroid_router = CentroidRouter(miniLM_embedder).fit(clusters_data)

#### test

In [241]:
# centroid_router.predict(['напиши сортировку пузырьком']), centroid_router.predict(['Где в можно вкусно пообедать']), centroid_router('Где в можно вкусно пообедать')

### Combined router

In [243]:
# class CombinedRouter:
#     def __init__(self, keyword_router, centroid_router):
#         self._keyword_router = keyword_router
#         self._centroid_router = centroid_router
        
#     def __call__(self, input_txt: str) -> str:
#         route = self._keyword_router(input_txt)
#         if route == self._keyword_router._default:
#             return self._centroid_router([input_txt])
#         return route
    
#     def __or__(self, input_txt: str) -> str:
#         return self(input_txt)
    
#     def __ror__(self, input_txt: str) -> str:
#         return self(input_txt)
    
# combined_router = CombinedRouter(keyword_router, centroid_router)

#### test

In [245]:
# (
#     combined_router('напиши сортировку пузырьком'), 
#     combined_router('Где в можно вкусно пообедать'),
# )

## Corrector

In [247]:
from langchain.chains.graph_qa import cypher_utils

class CypherQueryCorrector(Chainable):    
    def __init__(self, graph):
        self._graph = graph
        self._query_corrector = None
        self._set_or_update_query_corrector()
        

    def _set_or_update_query_corrector(self):
        corrector_schema = [
            cypher_utils.Schema(element["start"], element["type"], element["end"])
            for element in self._graph.structured_schema.get("relationships")
        ]
        self._query_corrector = cypher_utils.CypherQueryCorrector(corrector_schema)

    def __call__(self, query: str) -> str:
        logger.info(f'CypherQueryCorrector: {query=}')
        query = query.replace('`', '').strip()
        return self._query_corrector(query)

cypher_query_corrector = CypherQueryCorrector(graph)

#### test

In [249]:
# generated_cypher = '**match (n:Place {category: "food"})-[]-(t:Keyword {type_: "LOC", text: "санкт-петербург"}) return n**'
# generated_cypher = cypher_query_corrector(generated_cypher)
# generated_cypher

## Nodes Printer

In [251]:
class Neo4jPrinter(Chainable):
    def _parse_unsupported(self, vals: dict) -> str:
        return ''
        
    def _print_place(self, vals: dict) -> str:
        return f"{vals['title']}, находится по адресу: {vals['address']} \n"
    
    def _print_doc(self, vals: dict) -> str:
        text = vals['text']
        if len(text) > 2000:
            text = ''
        return f"{vals['title']}\n{text}"
    
    def _print_span(self, vals: dict) -> str:
        return f"{vals['text']}"
    
    def __call__(self, vals: dict) -> str:
        resp = ''
        for ent in vals:
            match ent:
                case {'address': _}:
                    resp += self._print_place(ent)
                case {'link': _}:
                    resp += self._print_doc(ent)
                case {'text': _}:
                    resp += self._print_span(ent)
                case _:
                    resp += self._parse_unsupported(ent)
        return resp

n4j_printer = Neo4jPrinter()

#### test

In [253]:
# print(n4j_printer(test_resp))

## Limit selector

In [255]:
class LimitSelector:
    def __init__(self, limit: int):
        self._limit = limit
        
    def __call__(self, nodes):
        return nodes[:self._limit]

## Relevance sorter

#### Sorter

In [258]:
config_params = {
    'raw': {
        'biased_rating': 100,
        'fill_score': 10,
        'review_count': 30, #20
        # 'worktime_is_24x7': 10, #5
        'is_accessable': 15, #5
        'is_modern': 10, #5
        'has_wifi': 15, #5
        'has_music': 5,
        'contact_social_len': 3,
        'phone_len': 2,
        'photos_len': 5,
        'has_website': 5,
    },
    'meta': {
        'category_weights': {
            'food': 7,
            'services': 6,
            'religious': 5,
            'shopping': 4,
            'recreation': 3,
            'entertainment': 2,
            'sports': 1,
        },
        'category': 10,
        'distance': 50,
    },
}

def opt_print(v, must):
    if must:
        print(v)

class RelevanceSorter(Chainable):
    def __init__(self, config):
        self.config = config

    def get_score(self, node, verbose=False):
        score = 0.0

        # считаем по прямым фичам в форе
        for param_name, weight in self.config['raw'].items():
            opt_print(f'{param_name} {node[param_name] * weight}', verbose)
            score += node[param_name] * weight
        opt_print(f'raw {score=}', verbose)

        # считаем оценку с учетом цели поездки
        category_score = self.config['meta']['category_weights'][node['category']] / 7
        category_score *= self.config['meta']['category']
        opt_print(f'{category_score=}', verbose)

        # итоговая оценка это оценка "прямых" фич, расстояния и веса категории
        return sum([score, category_score])
    
    def select(self, nodes: list[dict]):
        nodes = map(lambda n: {
            'score': self.get_score(n),
            **n,
        }, nodes)
        return sorted(nodes, reverse=True, key=lambda n: n['score'])

    def __call__(self, nodes: list[dict]):
        return self.select(nodes)
    
relevance_sorter = RelevanceSorter(config_params)

#### test

In [260]:
# relevance_sorter(test_resp), relevance_sorter | test_resp

## Utils

### Logger

In [263]:
# class InputLogger(Chainable):
#     def __call__(self, params):
#         logger.info(f'{params=}')
#         return params
    
# input_logger = InputLogger()

### Getter

In [265]:
class Getter(Chainable):
    def __init__(self):
        self.target = None
        
    def __init__(self, getter=None):
        self.target = None
        self.getter = getter
        
    def __call__(self, params):
        if not self.target:
            return self.getter(params)
        return self.target(self.getter(params))
    
    def __or__(self, target):
        self.target = target
        return self

### Map

In [267]:
class Map(Chainable):
    def __init__(self, key, runnable):
        self.runnable = runnable
        self.key = key
        
    def __call__(self, params):
        params[self.key] = self.runnable(params[self.key])
        return params

### Optional map

In [269]:
class OptionalMap:
    def __init__(self, key, runnable, criteria):
        self.key = key
        self.runnable = runnable
        self.criteria = criteria
        
    def __call__(self, params):
        if self.criteria(params):
            return self.runnable(params)
        return params[self.key]

### Nothing chain

In [271]:
class DoNothing(Chainable):
    def __call__(self, params):
        return params

### Response

In [273]:
# class BadResp:
#     def __call__(self, _):
#         return 'ops'

## Chain

In [275]:
llm = yallm
assigner = RunnablePassthrough.assign

#### fake llm

In [277]:
# class FakeQueryLLM(LLM):
#     @property
#     def _llm_type(self) -> str:
#         return "custom"

#     def _call(
#         self,
#         prompt: str,
#         stop: typing.Optional[list[str]] = None,
#         run_manager: typing.Optional[CallbackManagerForLLMRun] = None,
#         **kwargs: typing.Any,
#     ) -> str:
#         return test_cypher
    
# class FakeResponseLLM(LLM):

#     @property
#     def _llm_type(self) -> str:
#         return "custom"

#     def _call(
#         self,
#         prompt: str,
#         stop: typing.Optional[list[str]] = None,
#         run_manager: typing.Optional[CallbackManagerForLLMRun] = None,
#         **kwargs: typing.Any,
#     ) -> str:
#         return f'FakeResponseLLM: {prompt}'
    
# qllm = FakeQueryLLM()
# rllm = FakeResponseLLM()

### No data chain

In [279]:
no_data_chain = assigner(response=no_data_response_prompt | llm)

#### test

In [281]:
# q = 'где есть банкоматы в центре Санкт-Петербурга?'
# t = no_data_chain.invoke({
#     'input': q,
#     'question': q,
#     'tagged_question': 'где есть банкоматы в центре <LOC>санкт-петербург<LOC>?',
# })
# t

### Not travel chain

In [283]:
not_travel_chain = assigner(response=not_travel_response_prompt | llm)

#### test

In [285]:
# q = 'напиши сортировку пузырьком на C++'
# t = not_travel_chain.invoke({
#     'input': q,
#     'question': q,
#     'tagged_question': q,
# })
# t

### Preprocessing chain

In [287]:
preprocessing_chain = (
    assigner(question=Getter(itemgetter('input')))
    | assigner(entities=lambda p: spacy_extractor(p['question']))
    | assigner(keywords=lambda p: keyworder(p['entities']))
    | assigner(tagged_question=lambda p: tagger(p['question'],p['entities']))
    | assigner(embedds=Getter(itemgetter('question')) | input_embedder)
)

#### test

In [289]:
# q = 'Какое задание Данила получает от своего брата Виктора, также известного как Татарин?'
# t = preprocessing_chain.invoke({'input': q})
# t.keys()

In [290]:
# t['tagged_question']

### HyDE chain

In [292]:
def make_hyde_embeddings(answers_text, embedder):
    answers = answers_text.split(';')
    return [embedder(ans) for ans in answers]

In [293]:
hyde_chain = (
    assigner(answers=hyde_query_prompt | llm)
    | assigner(hyde_embeddings=lambda p: make_hyde_embeddings(p['answers'], input_embedder))
    | assigner(nodes=lambda p: vector_querier.query_many(p['hyde_embeddings']))
)

#### test

In [295]:
# test_chain = hyde_chain
# q = 'Какое задание Данила получает от своего брата Виктора, также известного как Татарин в фильме Брат?'
# t = test_chain.invoke({'question': q})
# t.keys()

In [296]:
# str(t['nodes'])[:100]

In [297]:
# t['answers'].split(';')

### Deepsearch

In [299]:
class FakeQuerier:
    def __init__(self, resp):
        self.resp = resp
        
    def __call__(self, *q):
        print(str(q)[:100])
        return self.resp
    
fake_q_0 = FakeQuerier([])
fake_q_1 = FakeQuerier([{}])

In [300]:
deepsearch_chain = (
    assigner(nodes=lambda p: hybrid_querier(p['embedds'], p['keywords']))
    | assigner(nodes=OptionalMap(
        'nodes',
        runnable=lambda p: vector_querier(p['embedds']),
        criteria=lambda p: len(p['nodes']) == 0,
    ))
    | RunnableBranch(
        (lambda v: len(v["nodes"]) == 0, hyde_chain),
        DoNothing(),
    )
)

#### test

In [302]:
# test_chain = preprocessing_chain | deepsearch_chain
# q = 'Какое задание Данила получает от своего брата Виктора, также известного как Татарин в фильме Брат?'
# t = test_chain.invoke({'input': q})
# t.keys()

In [303]:
# str(t['nodes'])[:100]

### Place chain

In [305]:
place_response_chain = (
    assigner(nodes=Getter(itemgetter('nodes'))| LimitSelector(10))
    | assigner(additional_info=Getter(itemgetter('nodes'))| n4j_printer)
    | assigner(response=places_response_prompt | llm)
)

place_response_branch = RunnableBranch(
    (lambda v: len(v["nodes"]) > 0, place_response_chain),
    no_data_chain,
)

place_chain = (
    assigner(generated_query=places_query_prompt | llm)
    | assigner(query=Getter(itemgetter('generated_query')) | cypher_query_corrector)
    | assigner(nodes=Getter(itemgetter('query')) | graph_querier)
    | assigner(nodes=Getter(itemgetter('nodes'))| relevance_sorter)
    | place_response_branch
)

#### test

In [307]:
# q = 'где хорошо поесть в Санкт-Петербурге?'
# t = place_chain.invoke({
#     'input': q,
#     'question': q,
#     'tagged_question': 'где хорошо поесть в <LOC>санкт-петербург<LOC>?',
#     'graph_schema': graph.schema, 
#     'options': PLACES_OPTIONS,
#     'example': TRIP_EXAMPLE,
# })
# t

In [308]:
# t['additional_info']

### Info chain

In [310]:
info_response_chain = (
    assigner(nodes=Getter(itemgetter('nodes'))| LimitSelector(3))
    | assigner(additional_info=Getter(itemgetter('nodes'))| n4j_printer)
    | assigner(response=info_response_prompt | llm)
)

info_response_branch = RunnableBranch(
    (lambda v: len(v["nodes"]) > 0, info_response_chain),
    no_data_chain,
)

info_chain = (
    deepsearch_chain
    | info_response_branch
)

#### test

In [312]:
# test_chain = preprocessing_chain | info_chain
# q = 'Какое задание Данила получает от своего брата Виктора, также известного как Татарин в фильме Брат?'
# t = test_chain.invoke({'input': q})
# t

### Trip chain

In [314]:
trip_chain = (
    assigner(nodes=lambda _: graph_querier(trip_query))
    | assigner(nodes=Getter(itemgetter('nodes'))| relevance_sorter)
    | assigner(nodes=Getter(itemgetter('nodes'))| LimitSelector(10))
    | assigner(additional_info=Getter(itemgetter('nodes'))| n4j_printer)
    | assigner(response=trip_response_prompt | llm)
)

#### test

In [316]:
# q = 'спланируй поездку в Санкт-Петербурге'
# t = trip_chain.invoke({
#     'input': q,
#     'question': q,
#     'tagged_question': 'спланируй поездку в <LOC>санкт-петербург<LOC>',
# })
# t.keys()

### Full

In [318]:
routes_branch = RunnableBranch(
    (lambda v: v["topic"] in ["place", 'city_places'], place_chain),
    (lambda v: v["topic"] in ["info", 'travel_info', 'movie_storyline', 'movie_places', 'travel_place_info'], info_chain),
    (lambda v: v["topic"] in ["trip"], trip_chain),
    not_travel_chain,
)

chain = (
    assigner(question=Getter(itemgetter('input')))
    | preprocessing_chain
    | assigner(topic=Getter(itemgetter('question')) | centroid_router)
    | routes_branch
    | Getter(itemgetter('response'))
)

def run(q):
    try:
        return chain.invoke({
            'input': q,
            'graph_schema': graph.schema, 
            'options': PLACES_OPTIONS,
        })
    except Exception as e:
        return str(e)

#### test

In [320]:
# template = "{question}"
# prompt = PromptTemplate.from_template(template)
# llm_chain = LLMChain(prompt=prompt, llm=llm)

# def runllm(q):
#     try:
#         question = q
#         return llm_chain.invoke(question)
#     except Exception as e:
#         return str(e)

In [321]:
# q = 'Какое задание Данила получает от своего брата Виктора, также известного как Татарин?'
# # q = 'где хорошо поесть в Санкт-Петербурге?'
# t = run(q)
# #t = runllm(q)

In [322]:
# t['topic']

# Tests

In [324]:
def print_results(results):
    before = '------------------------------------------------------------------------'
    after = '========================================================================'
    for q, a in results.items():
        text = a if issubclass(str, a.__class__) else a["response"]
        print(f'{before}\n{q}:\n\n{text}\n{after}\n\n')

## Embeds

In [326]:
# title = 'расскажи про Музей «Павловские Колтуши»'
# test_embeds = input_embedder.embed(title)
# test_embeds[:10]

In [327]:
# doc_embeddings_query = '''
# CALL db.index.vector.queryNodes('doc_embeddings', $limit, $embeddings) 
# YIELD node, score
# MATCH (node)
# RETURN 
#     node.title, 
#     score
# ORDER BY score DESC 
# '''

# span_embeddings_query = '''
# CALL db.index.vector.queryNodes('doc_embeddings', $limit, $embeddings) 
# YIELD node, score
# MATCH (node)
# RETURN 
#     node.title, 
#     score
# ORDER BY score DESC 
# '''

# place_embeddings_query = '''
# CALL db.index.vector.queryNodes('doc_embeddings', $limit, $embeddings) 
# YIELD node, score
# MATCH (node)
# RETURN 
#     node.title, 
#     score
# ORDER BY score DESC 
# '''

In [328]:
# graph_querier.query(doc_embeddings_query, params={
#     'embeddings': test_embeds,
#     'limit': 10,
# })
# # print(query_text)
# # conn.query(query_text, parameters)#, test_embeds

## Light

In [330]:
results = {q: run(q) for q in [
    'Спланируй поездку в Санкт-Петербурге',
    'Где можно отдохнуть в Санкт-Петербурге?',
    'Где развлечься в Санкт-Петербурге?',
    'Где происходят события фильма "Брат"?',
    'Расскажи про Спас на Крови',
    'Расскажи про Апраксин двор',
    'Расскажи про «Подписные издания»',
    'Расскажи про Павловск',
    'напиши сортировку пузырьком на C++',
]}

GraphQuerier: 
match (n:Place)
where
n.category in...", "shopping"]
return n
, kwargs={}, len(resp)=168

# Context
I have data about places. I need to ans... 

# Question
Спланируй поездку в Санкт-Петербурге
Task:Generate Cypher statement to query a graph da...n
Где можно отдохнуть в <LOC>санкт-петербург<LOC>?
CypherQueryCorrector: query='match (n:Place {categ...{type_: "LOC", text: "санкт-петербург"}) return n'
GraphQuerier: match (n:Place {category: "recreatio...нкт-петербург"}) return n, kwargs={}, len(resp)=42

# Context
I have data about places. I need to ans...# Question
Где можно отдохнуть в Санкт-Петербурге?
Task:Generate Cypher statement to query a graph da...estion
Где развлечься в <LOC>санкт-петербург<LOC>?
CypherQueryCorrector: query='```\nmatch (n:Place {..._: "LOC", text: "санкт-петербург"}) return n\n```'
GraphQuerier: match (n:Place {category: "entertain...нкт-петербург"}) return n, kwargs={}, len(resp)=42

# Context
I have data about places. I need to ans...54 

# Ques

In [331]:
# results['где сходить в магазин в Питере?']

In [472]:
print_results(results)

------------------------------------------------------------------------
Спланируй поездку в Санкт-Петербурге:

**День 1**

Утро: Начните день с завтрака в Baggins Coffee — уютной кофейне, где можно насладиться чашечкой кофе и выпечкой.

День: Посетите культурно-досуговый центр Красногвардейский, чтобы узнать больше о культуре и истории города.

Вечер: Устройте ужин в ресторане Jager Meet, который славится своей мясной кухней.

**День 2**

Утро: Проведите утро, прогуливаясь по торгово-развлекательному комплексу «Лето», где можно сделать покупки и пообедать в одном из кафе.

День: Отдохните в парке или саду, наслаждаясь весенним солнцем и свежим воздухом.

Вечер: Насладитесь ужином в душевном локальном баре Queens.

**День 3**

Утро: Посетите пекарню «Кап&Кейк» и попробуйте свежую выпечку.

День: Проведите время в одном из музеев или галерей Санкт-Петербурга.

Вечер: Завершите день ужином в ресторане грузинской кухни Hurma.

Для каждого дня можно предусмотреть альтернативные варианты, у

In [333]:
# 'В какие годы происходят события фильма "Брат"?

## Benchmark

In [335]:
# brat_storyline = [
#     'В какие годы происходят события фильма "Брат"?',
#     'Куда в самом начале приезжает Данила Багров?',
#     'Что стало причиной задержания Данила Багрова в отделении милиции?',
#     'Какое задание Данила получает от своего брата Виктора, также известного как Татарин?',
#     'Какие обстоятельства привели к знакомству Данилы с Гофманом в Санкт-Петербурге?',
#     'Какой поворот сюжета происходит после убийства Чечена по заказу Виктора?',
#     'Какие действия предпринимает Данила для спасения Светы и как это влияет на их отношения?',
#     'Куда уезжает Данила в конце?',
#     'Чем заканчивается история Данилы и его брата Виктора в фильме?',
#     'Какова роль музыки и определенных песен в развитии сюжета и характере главного героя?',
# ]
# brat_places = [
#     'расскажи в каких местах в Санкт-Петербурге были съемки Брата?',
#     'расскажи какие памятники в Санкт-Петербурге попали в фильма съемки "Брат"?',
#     'расскажи через какие знаковые места Санкт-Петербурга проходит путь Данилы Багрова в эпизоде с грузовым трамваем?',
#     'Где живет брат Данилы?',
#     'расскажи про места съемок фильма Брат',
# ]
# city_places = [
#     'где сходить в магазин в Питере?',
#     'какие рестораны стоит посетить в Санкт-Петербурге?',
#     'где хорошо поесть в Санкт-Петербурге?',
#     'где можно отдохнуть в Санкт-Петербурге?',
#     'где развлечься в Санкт-Петербурге?',
# ]
# travel_info = [
#     'расскажи про мост со скульптурами «Укрощение коня»',
#     'расскажи про конструктивистское здание Научного городка в Санкт-Петербурге',
#     'расскажи про музей «Павловские Колтуши»',
#     'расскажи про Юсуповский дворец',
#     'расскажи где вид с высоты на Петербург',
#     'расскажи что такое Стрелка в Петербурге',
#     'расскажи когда Невский проспект обрёл нынешний вид',
#     'расскажи про атмосферный книжный магазин в Санкт-Петербурге',
#     'расскажи про «Подписные издания»',
#     'расскажи где было совершено седьмое покушение на Александра II',
#     'расскажи про Спас на Крови',
#     'расскажи про Павловский пейзажный парк',
#     'расскажи про какойнибудь большой парк в стране',
#     'расскажи про Мечеть в Петербурге',
#     'расскажи что-нибудь про Ладожское озеро',
#     'расскажи про Елагин остров',
#     'расскажи про Петропавловская крепость',
#     'расскажи про Толстовский дом',
#     'расскажи про Апраксин двор',
#     'расскажи про Павловск',
# ]
# trip = ['спланируй поездку в Санкт-Петербурге']
# bad = ['напиши сортировку пузырьком на C++']


# benchmark_questions = brat_storyline + brat_places + city_places + travel_info + trip + bad
# benchmark_questions

# print(f'''
# {len(benchmark_questions)=} 
# {len(brat_storyline)=} 
# {len(brat_places)=} 
# {len(city_places)=} 
# {len(travel_info)=} 
# {len(trip)=} 
# {len(bad)=}
# ''')

In [336]:
# results = {q: runllm(q) for q in benchmark_questions}

In [337]:
# results = {q: run(q) for q in benchmark_questions}

In [338]:
# t = '''
# '''

In [339]:
# len([v for v in t.split(' ') if 'len(' in v])

In [340]:
# def print_retrive_res(results):
#     for q, r in results.items():
#         print(q)
#         if issubclass(dict, r.__class__):
#             # print(r.keys())
#             if issubclass(list, r['query'].__class__):
#                 print(r['query'][:5])
#                 print(r['topic'])
#                 print(r['nodes'][0])
#             else:
#                 print(r['query'])
#         else:
#             print(r)
#         print('----------------')

# print_retrive_res(results)

### Final

In [342]:
# import datetime

In [343]:
# date = datetime.datetime.now().strftime('%y%m%d')
# res_df = pd.DataFrame([[k,v] for k, v in results.items()], columns=['question', 'answer'])
# res_df.to_excel(f'../bench-results/bech_res_{date}_base.xlsx')  
# res_df